In [ ]:
# 1)    import modules and packages

import requests #get url
from bs4 import BeautifulSoup #parse
import json
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 2)  create url and scrape single game shots

base_url = "https://understat.com/match/"
match_id =  str(input('Enter Match ID:'))
url = base_url + match_id

In [ ]:
# 3)    Use request and BeautifulSoup to scrape and parse data

res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml') #inspect page - sources - look at the html and select the scripts with variable you want to parse next
scripts = soup.find_all('script') # find all the scripts in the page

In [ ]:
#   get shots data only

strings = scripts[1].string # her script tag list bir list item, ShotsData bu element listteki ikinci eleman then convert into string

#   strip the unnecessary bits

index_start = strings.index("('")+2 # start stripping 2 digits after the first (' encountered
index_end = strings.index("')") # end stripping after the ') sequence is encountered

json_data = strings[index_start:index_end]
json_data = json_data.encode('utf8').decode('unicode_escape')

#   convert strings to json format

data = json.loads(json_data)
#data

In [ ]:
# 4)    convert to JSON and a df using pandas

team = []
minute = []
x = []
y = []
xG = []
data_home = data['h']
data_away = data['a']
player = []
result = []
hora = []

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'minute':
            minute.append(data_home[index][key])
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'player':
            player.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])
        if key == 'h_a':
            hora.append(data_home[index][key])
        

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'minute':
            minute.append(data_away[index][key])   
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'player':
            player.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])
        if key == 'h_a':
            hora.append(data_away[index][key])

In [ ]:
# create the dataframe

col_names = [ 'team', 'minute', 'x', 'y', 'xG', 'player', 'result', 'hora']
df = pd.DataFrame([team,minute,x,y,xG,player,result,hora], index = col_names)
df = df.T # transpose

df['x'] = pd.to_numeric(df['x'])
df['y'] = pd.to_numeric(df['y'])
df['xG'] = pd.to_numeric(df['xG']) #convert xG into numeric for bubble size

df.loc[df['hora'] == 'h', 'hora'] = 0 # if home rep w/ zero
df.loc[df['hora'] == 'a', 'hora'] = 1 # if away rep with one

In [ ]:
df

In [ ]:
#team_identifier = df.team.unique()
#Home_Team = team_identifier[0]
#Away_Team = team_identifier[1]

In [ ]:
#df['xG'] = 100*(df['xG']) 
ShotChart = plt.scatter(df.x, df.y, s = 1000*df.xG, c = df.hora)
plt.xlim([0.8,1])
plt.ylim([1,0])

#label = f"{df['player']}, {df['xG']}"
#plt.annotate(label, (df['x'], df['y']), textcoords="offset points")